# Filter curated haplotypes based on the read counts matrix
- in each locus, drop an individual from it if there are fewer than a certain number of reads
- drop individuals entirely if too much missing data

## figure out which individuals to drop from loci, and which to drop completely

In [3]:
import collections
import json
import numpy as np
import pandas as pd
read_counts = pd.read_csv("../haplotype_pipeline/results/read_counts.csv", index_col=0)
read_counts.head()

,1299,1319,1524,1531,1536,1541,1546,1547,1688,1779,...,482,58,59,6934,8911,DRL,OUT356,OUT366,SCW027,SCW028
E10A7,274,244,464,6,10,5,829,1096,2,968,...,455,218,106,134,100,266,20,259,156,227
E10C11,598,494,1873,130,258,2393,2499,1216,1,6519,...,1415,443,254,126,32,172,34,480,306,200
E10C5,640,786,69,73,264,1568,1924,3110,0,6265,...,858,1367,727,39,246,326,126,805,432,792
E10C6,185,378,1245,88,0,480,945,1508,0,3410,...,566,443,126,136,82,24,22,170,110,153
E10E9,793,76,729,44,4,231,1677,2162,0,3172,...,90,99,46,149,30,128,28,220,155,92


In [15]:
read_count_threshold = 5
loci_missing_threshold = 0.5
num_loci = len(read_counts.index)
read_counts_filtered = read_counts.copy()

indivs_to_drop_from_loci = collections.defaultdict(list)  # indiv: set{locus1, locus2...}
for locus in read_counts.index:
    for indiv_id in read_counts.columns:
        count = read_counts.at[locus, indiv_id]
        if count < read_count_threshold:
            indivs_to_drop_from_loci[indiv_id].append(locus)
            read_counts_filtered.at[locus, indiv_id] = 0
print(len(indivs_to_drop_from_loci))
            
indivs_to_drop_all = [indiv for indiv in indivs_to_drop_from_loci 
                      if len(indivs_to_drop_from_loci[indiv]) >= loci_missing_threshold * num_loci]
print(len(indivs_to_drop_all))

for indiv in indivs_to_drop_all :
    indivs_to_drop_from_loci.pop(indiv)
read_counts_filtered = read_counts.drop(list(indivs_to_drop_all), axis=1)
print(len(indivs_to_drop_from_loci))

220
13
207


## write results

In [82]:
with open("snp_pipeline/results/indivs_to_drop_from_loci.json", "w") as out_file:
    json.dump(indivs_to_drop_from_loci, out_file)
with open("snp_pipeline/results/indivs_to_drop_all.json", "w") as out_file:
    json.dump(indivs_to_drop_all, out_file)

## filter curated haplotypes files based on rules set above

testing with locus E10A7

In [ ]:
import Bio.SeqIO
locus = "E10A7"
locus_file = Bio.SeqIO.parse(f"snp_pipeline/haplotypes_curated/{locus}.fna", "fasta")
filtered_seqs = [seq for seq in locus_file if (seq.id.split('_')[0] not in indivs_to_drop_all) and (locus not in indivs_to_drop_from_loci[seq.id.split('_')[0]])]
Bio.SeqIO.write(filtered_seqs, f"snp_pipeline/haplotypes_filtered/{locus}.fna", "fasta")

## test json loading

In [89]:
with open("snp_pipeline/results/indivs_to_drop_from_loci.json", "r") as in_file:
    indivs_to_drop_all = {key: set(value) for key, value in json.load(in_file).items()}
with open("snp_pipeline/results/indivs_to_drop_all.json", "r") as in_file:
    indivs_to_drop_from_loci = set(json.load(in_file))